#### Trying out DensNet121

In [1]:
from PIL import Image
from torchvision import transforms, models
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
from torch import optim
import torch.nn.init as init


from tqdm.notebook import tqdm

In [2]:
class MyDenseNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.densenet = models.densenet121(pretrained=True)
        num_features = self.densenet.classifier.in_features #access the number of input features for last fully connected layer
        #replace last fully connected layer with linear layer that matches no of output classes
        self.densenet.classifier = nn.Linear(num_features, num_classes)

        # Apply Xavier initialization to the linear layer's weights
        init.xavier_uniform_(self.densenet.classifier.weight)
        if self.densenet.classifier.bias is not None:
            init.constant_(self.densenet.classifier.bias, 0)  # Initialize bias if present

    def forward(self, x):
        return self.densenet(x)

In [3]:
num_classes = 3  # no of classes for classification
model = MyDenseNet(num_classes) # initialize our densnet model with the number of classes

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 182MB/s]


 As our model still hasn't been trained yet it has not yet learned any patterns from the training data. This code is just to see if the model loading is working with a bunch of images that we have extracted. Here we are making predictions with an untrained model for more of a demonstration of the prediction process itself.

#### Spliting into Train, Test and Val

In [4]:
import random
import shutil
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [5]:
classes = [class_name for class_name in os.listdir('dataset') if os.path.isdir(os.path.join('dataset', class_name))]
print(classes)

FileNotFoundError: ignored

In [6]:
data_folder = 'dataset/C57BL_6J' # this is only for one type, make it dynamic for looping all types of cells
train_folder = 'dataset/train_folder'
val_folder = 'dataset/val_folder'
test_folder = 'dataset/test_folder'

os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

In [ ]:
image_files = os.listdir(data_folder)
random.shuffle(image_files)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/C57BL_6J'

In [ ]:
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

total_images = len(image_files)
train_count = int(total_images * train_ratio)
val_count = int(total_images * val_ratio)

train_files = image_files[:train_count]
val_files = image_files[train_count:train_count + val_count]
test_files = image_files[train_count + val_count:]

NameError: name 'image_files' is not defined

In [ ]:
for filename in train_files:
    shutil.move(os.path.join(data_folder, filename), os.path.join(train_folder, filename))

for filename in val_files:
    shutil.move(os.path.join(data_folder, filename), os.path.join(val_folder, filename))

for filename in test_files:
    shutil.move(os.path.join(data_folder, filename), os.path.join(test_folder, filename))

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageFolder('dataset/train_folder', transform=transform)
val_dataset = ImageFolder('dataset/test_folder', transform=transform)
test_dataset = ImageFolder('dataset/val_folder', transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
loss_fn = nn.CrossEntropyLoss() # We use Cross Entropy Loss, as this is a classification task
#optimizer = optim.Adam(model.parameters(), lr=0.001) # If in doubt, we use Adam as our optimiser
optimizer = optim.SGD(model.parameters(), lr=0.001) # If in doubt, we use Adam as our optimiser
from torch.optim.lr_scheduler import StepLR

milestones = [10, 18, 24]  # List of epochs where the learning rate will be decreased
gamma = 0.1  # Factor by which the learning rate will be decreased
scheduler = StepLR(optimizer, step_size=10, gamma=gamma)  # Adjust step_size according to your preference


In [ ]:
def train(model, train_loader, loss_fn, optimizer, device):
    model.train() # puts the model in training mode
    running_loss = 0
    with tqdm(total=len(train_loader)) as pbar:
        for i, data in enumerate(train_loader, 0): # loops through training data
            inputs, labels = data # separate inputs and labels (outputs)
            inputs, labels = inputs.to(device), labels.to(device) # puts the data on the GPU

            # forward + backward + optimize
            optimizer.zero_grad() # clear the gradients in model parameters
            outputs = model(inputs) # forward pass and get predictions
            loss = loss_fn(outputs, labels) # calculate loss
            loss.backward() # calculates gradient w.r.t to loss for all parameters in model that have requires_grad=True
            optimizer.step() # iterate over all parameters in the model with requires_grad=True and update their weights.

            running_loss += loss.item() # sum total loss in current epoch for print later

            pbar.update(1) #increment our progress bar

    return running_loss/len(train_loader) # returns the total training loss for the epoch

In [ ]:
# Function for the validation pass

def validation(model, val_loader, loss_fn, device):
    model.eval() # puts the model in validation mode
    running_loss = 0
    total = 0
    correct = 0

    with torch.no_grad(): # save memory by not saving gradients which we don't need
        with tqdm(total=len(val_loader)) as pbar:
            for images, labels in iter(val_loader):
                images, labels = images.to(device), labels.to(device) # put the data on the GPU
                outputs = model(images) # passes image to the model, and gets a ouput which is the class probability prediction

                val_loss = loss_fn(outputs, labels) # calculates val_loss from model predictions and true labels
                running_loss += val_loss.item()
                _, predicted = torch.max(outputs, 1) # turns class probability predictions to class labels
                total += labels.size(0) # sums the number of predictions
                correct += (predicted == labels).sum().item() # sums the number of correct predictions

                pbar.update(1)

        return running_loss/len(val_loader), correct/total # return loss value, accuracy

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Determine whether a GPU is available
model.to(device) # send model to GPU

MyDenseNet(
  (densenet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum

In [ ]:
total_epoch = 30 # Define how many epochs of training we want

# keep track of things we'd like to plot later
training_losses = []
validation_losses = []
accuracies = []

for epoch in range(total_epoch): # loops through number of epochs
    train_loss = train(model, train_loader, loss_fn, optimizer, device)  # train the model for one epoch
    # Update the learning rate scheduler
    scheduler.step()
    val_loss, accuracy = validation(model, val_loader, loss_fn, device) # after training for one epoch, run the validation() function to see how the model is doing on the validation dataset

    # keep track of interesting stuff
    training_losses.append(train_loss)
    validation_losses.append(val_loss)
    accuracies.append(accuracy)

    print("Epoch: {}/{}, Training Loss: {}, Val Loss: {}, Val Accuracy: {}".format(epoch+1, total_epoch, train_loss, val_loss, accuracy))
    print('-' * 20)

print("Finished Training")

# Save the queen
torch.save(model.state_dict(), 'finished')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 1/30, Training Loss: 0.3377733826637268, Val Loss: 1.091655969619751, Val Accuracy: 0.4444444444444444
--------------------


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 2/30, Training Loss: 0.13250131905078888, Val Loss: 1.2508097887039185, Val Accuracy: 0.4444444444444444
--------------------


  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 